In [13]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import string
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

# import different metrics to evaluate the classifiers
from sklearn.metrics import accuracy_score

# from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics

# import time function from time module to track the training duration
from time import time

In [5]:
#@ Loading dataset:
try:
    from google.colab import files
    !wget -P DATAPATH https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Full-Economic-News-DFE-839861.csv
    !ls -lah DATAPATH
    our_data = pd.read_csv("DATAPATH/Full-Economic-News-DFE-839861.csv" , encoding = "ISO-8859-1" )

except ModuleNotFoundError:
    our_data = pd.read_csv("Data/Full-Economic-News-DFE-839861.csv" , encoding = "ISO-8859-1" )


--2024-11-27 15:00:44--  https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Full-Economic-News-DFE-839861.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12383529 (12M) [application/octet-stream]
Saving to: ‘DATAPATH/Full-Economic-News-DFE-839861.csv’

Full-Economic-News- 100%[===================>]  11.81M  --.-KB/s    in 0.04s   

2024-11-27 15:00:44 (296 MB/s) - ‘DATAPATH/Full-Economic-News-DFE-839861.csv’ saved [12383529/12383529]

total 12M
drwxr-xr-x 2 root root 4.0K Nov 27 15:00 .
drwxr-xr-x 1 root root 4.0K Nov 27 15:00 ..
-rw-r--r-- 1 root root  12M Nov 27 15:00 Full-Economic-News-DFE-839861.csv


In [9]:
display(our_data.shape)
our_data['relevance'].value_counts()/our_data.shape[0]

(8000, 15)

,count
relevance,
no,0.821375
yes,0.177500
not sure,0.001125


In [10]:
#@ Converting label into numerical values:
our_data=our_data[our_data.relevance != 'not sure'] #removing
our_data['relevance']=our_data.relevance.map({'yes':1, 'no':0})
our_data=our_data[['text', 'relevance']] #taking only two columns

In [12]:
#@ Text preprocessing:
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS


stopwords=ENGLISH_STOP_WORDS
def clean(doc):
  doc=doc.replace("</br>", "")
  doc=''.join([char for char in doc if char not in string.punctuation and not char.isdigit()])
  doc=''.join([token for token in doc.split() if token not in stopwords])
  return doc

In [19]:
#@ Modelling:
import sklearn
from sklearn.model_selection import train_test_split

X=our_data.text
y=our_data.relevance

X_train, X_test, y_train, y_test=train_test_split(X, y, random_state=33)

# preprocessing of vectorize train and test data
vect=CountVectorizer(preprocessor=clean)
X_train_dtm=vect.fit_transform(X_train)

X_test_dtm=vect.transform(X_test)

# training classifier:
nb=MultinomialNB()
%time nb.fit(X_train_dtm, y_train)
y_pred_class=nb.predict(X_test_dtm)
print("Accuracy:", accuracy_score(y_test, y_pred_class))

CPU times: user 5.1 ms, sys: 0 ns, total: 5.1 ms
Wall time: 5.11 ms
Accuracy: 0.8193193193193193


In [22]:
#@ with hyperparameter tuning:
vect=CountVectorizer(preprocessor=clean, max_features=5000)
X_train_dtm=vect.fit_transform(X_train)

X_test_dtm=vect.transform(X_test)

# training classifier:
nb=MultinomialNB()
%time nb.fit(X_train_dtm, y_train)
y_pred_class=nb.predict(X_test_dtm)
print("Accuracy:", accuracy_score(y_test, y_pred_class))


CPU times: user 2.88 ms, sys: 0 ns, total: 2.88 ms
Wall time: 2.89 ms
Accuracy: 0.8198198198198198
